In [1]:
import pandas as pd
import numpy as np

In [2]:
# df  = pd.read_csv('../data/hawkes_analysis_data.csv')
df  = pd.read_csv(r'I:/Hawkes_Model_MM/hawkes_analysis_data.csv')

In [3]:
df.head()

,trade_number,trade_price,volume,buy_order_number,buy_algo,buy_client,sell_order_number,sell_algo,sell_client,timestamp,buy_entry_ts,sell_entry_ts,aggressor_side,date
0,2019081325008066,785.0,1,1100000000068964,1,2,1100000000062580,1,3,2019-08-13 09:15:00.006118774,2019-08-13 09:15:00.005767822,NaN,-1,2019-08-13
1,2019081325008128,785.0,2,1100000000069441,1,3,1100000000062580,1,3,2019-08-13 09:15:00.024169922,2019-08-13 09:15:00.024139404,NaN,-1,2019-08-13
2,2019081325008129,785.0,47,1100000000069442,1,3,1100000000062580,1,3,2019-08-13 09:15:00.024230957,2019-08-13 09:15:00.024215698,NaN,-1,2019-08-13
3,2019081325008130,785.0,53,1100000000069442,1,3,1100000000068983,0,3,2019-08-13 09:15:00.024246216,2019-08-13 09:15:00.024215698,2019-08-13 09:15:00.005935669,1,2019-08-13
4,2019081325008466,785.0,10,1100000000055032,1,3,1100000000070554,0,1,2019-08-13 09:15:00.118347168,NaN,2019-08-13 09:15:00.118331909,1,2019-08-13


In [4]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [5]:
dates = df['date'].unique()

In [6]:
target_date = dates[0]

In [7]:
# df_day = df[df['date'] == target_date].sort_values('timestamp')
df_day = df[df['date'] == target_date].sort_values(['timestamp', 'trade_number'])

In [8]:
df_day['timestamp'].dtype

dtype('<M8[ns]')

In [9]:
market_open = pd.to_datetime((target_date) + ' 09:15:00')
df_day['t'] = (df_day['timestamp'] - market_open).dt.total_seconds()

In [10]:
buy_times = df_day[df_day['aggressor_side'] == +1]['t'].to_numpy()
sell_times = df_day[df_day['aggressor_side'] == -1]['t'].to_numpy()

events = [buy_times, sell_times]

In [11]:
decay = 1/60 

In [12]:
from BivariateHawkes_optimized import BivariateHawkesOptimized

Numba not available. Install with: pip install numba


In [13]:
model = BivariateHawkesOptimized()

In [ ]:
model.fit(buy_times, sell_times, optimize_beta=True)

In [16]:
results = model.get_results()

In [17]:
results

{'mu_buy': 1e-06,
 'mu_sell': 1e-06,
 'alpha_bb': 10.0,
 'alpha_bs': 10.0,
 'alpha_sb': 10.0,
 'alpha_ss': 10.0,
 'beta': 100.0,
 'br_bb': 0.1,
 'br_bs': 0.1,
 'br_sb': 0.1,
 'br_ss': 0.1,
 'spectral_radius': 0.2,
 'total_branching': 0.4,
 'log_likelihood': 602041.1113653451,
 'is_stationary': True}

In [18]:
buy_times

array([2.42462160e-02, 1.18347168e-01, 3.17398071e-01, ...,
       2.42857539e+04, 2.42857539e+04, 2.42857539e+04])

In [19]:
sell_times

array([6.11877400e-03, 2.41699220e-02, 2.42309570e-02, ...,
       2.24958606e+04, 2.24961908e+04, 2.24998983e+04])

In [23]:
df_day['t'] - df_day['t'].lag(1)

AttributeError: 'Series' object has no attribute 'lag'